In [1]:
import os
import shutil
import collections

import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt

import invisible_cities.core.fit_functions as fitf

In [ ]:
print(1)

In [3]:
%matplotlib inline

plt.rcParams["figure.figsize"] = 12, 8
plt.rcParams[  "font.size"   ] = 15

In [4]:
path_tables       = "/Users/Gonzalo/github/NEXTdata/NEWtables/"
filename_cathode  = os.path.join(path_tables,       "Cathode.dat")
filename_anode    = os.path.join(path_tables, "Anode_reduced.dat")
filename_pmt_map  = os.path.join(path_tables,   "pmt_mapping.dat")
filename_sipm_map = os.path.join(path_tables,  "sipm_mapping.dat")
filename_el_map   = os.path.join(path_tables,    "el_mapping.dat")

In [12]:
fix_file_cathode(filename_cathode)
fix_file_anode  (filename_anode  )

Reading /Users/Gonzalo/github/NEXTdata/NEWtables/Cathode.dat...
Reading /Users/Gonzalo/github/NEXTdata/NEWtables/Anode.dat...


In [14]:
pmt  = pd.DataFrame.from_csv(filename_pmt_map , sep=" ", index_col=0)
sipm = pd.DataFrame.from_csv(filename_sipm_map, sep=" ", index_col=0)
el   = pd.DataFrame.from_csv(filename_el_map  , sep=" ", index_col=0)

df_cathode = load_df_cathode(filename_cathode, el)
df_anode   = load_df_anode  (filename_anode  , el, sipm)

Reading /Users/Gonzalo/github/NEXTdata/NEWtables/Anode.dat...


In [8]:
assert np.nan not in df_cathode
assert np.nan not in df_anode.dR
assert np.nan not in df_anode.dRR

ValueError: cannot convert float NaN to integer

In [ ]:
df_anode.head()

# Control plots

In [ ]:
plt.scatter(pmt.X, pmt.Y, s=0)
for row_no, row in pmt.iterrows():
    plt.text(row.X, row.Y, str(int(row.SensorID)))
plt.xlabel("x (mm)")
plt.ylabel("y (mm)")
plt.title ("PMT positions");

In [ ]:
plt.scatter(sipm.X, sipm.Y, s=0)
for dice_no, dice in sipm.groupby("Dice"):
    x = dice.X.mean()
    y = dice.Y.mean()
    plt.text(x, y, str(int(dice_no)))
plt.xlabel("x (mm)")
plt.ylabel("y (mm)")
plt.title ("Dice positions");

In [ ]:
for dice_no, dice in sipm.groupby("Dice"):
    xmean = dice.X.mean()
    ymean = dice.Y.mean()
    plt.scatter(dice.X - xmean, dice.Y - ymean, s=0)
    for row_no, row in dice.iterrows():
        plt.text(row.X - xmean, row.Y - ymean, str(int(row.SensorID - 1000 * dice_no)))
    break
plt.xlabel("x (mm)")
plt.ylabel("y (mm)")
plt.title ("SiPM positions within dice");

In [ ]:
x_binning = np.arange(np.min(el.X), np.max(el.X) + 1)
y_binning = np.arange(np.min(el.Y), np.max(el.Y) + 1)
plt.hist2d(el.X, el.Y, (x_binning, y_binning), cmap="Greens")
plt.xlabel("x (mm)")
plt.ylabel("y (mm)")
plt.title ("EL points positions");

In [ ]:
bins = np.linspace(np.min(el.PointID), np.max(el.PointID), 201)

weights = np.ones(len(el)) / len(el)
plt.hist(el.PointID, bins=bins, weights=weights, color="k", histtype="step", lw=3)

weights = np.ones(len(df_anode)) / len(df_anode)
plt.hist(df_anode.PointID, bins=bins, weights=weights, color="m", histtype="step", lw=3)

plt.xlabel("EL point ID")
plt.ylabel("")
plt.title ("EL point ID distribution");

In [ ]:
bins    = np.linspace(0, np.max(df_cathode.Sum), 201)
weights = np.ones(len(df_cathode)) / len(df_cathode)
plt.hist(df_cathode.Sum, bins = bins, weights = weights, color="k", histtype="step", lw=3)
plt.xlabel("Total photon detection probability")
plt.ylabel("Fraction of events")

In [ ]:
bins    = np.linspace(0, max(np.max(df_anode.P0), np.max(df_anode.P1)), 201)
weights = np.ones(len(df_anode)) / len(df_anode)

plt.hist(df_anode.P0, bins = bins, weights = weights, color="k", histtype="step", lw=3, label="First  bin")
plt.hist(df_anode.P1, bins = bins, weights = weights, color="m", histtype="step", lw=3, label="Second bin")

plt.legend()
plt.xlabel("Photon detection probability")
plt.ylabel("Fraction of events")
plt.yscale("log")

# PMT parametrization

In [ ]:
def find_pitch(data):
    return np.min(np.diff(np.sort(np.unique(data))))

def find_nbins(data, d):
    return int(np.ceil((data.max() - data.min()) / d)) + 2

def find_binning(data):
    d = find_pitch(data)
    n = find_nbins(data, d)
    return np.linspace(df_cathode.X.min() - d/2,
                       df_cathode.X.max() + d/2,
                       n)

In [ ]:
x_binning = find_binning(df_cathode.X)
y_binning = find_binning(df_cathode.Y)
r_binning = np.linspace(0, np.max(df_cathode.R  ) * 1.001, 201)
p_binning = np.linspace(0, np.max(df_cathode.Sum) * 1.001, 201)

In [ ]:
plt.hist2d(df_cathode.X, df_cathode.Y,
           (x_binning, y_binning),
           weights = df_cathode.Sum / 12,
           cmin    = df_cathode.Sum.min(),
           cmax    = df_cathode.Sum.max(),
           cmap    = "jet"
          )

plt.colorbar()

In [ ]:
parametrizations = {}
profiles         = {}
pulls            = {}
for pmt_no, pmt in df_cathode.groupby("SensorID"):
    x, y, u = fitf.profileX(pmt.R, pmt.Sum, r_binning.size - 1, (r_binning[0], r_binning[-1]))
    ok = u > 0
    f  = fitf.fit(fitf.polynom, x[ok], y[ok], (-1, +1, -1, +1, -1, +1, pmt.Sum.max()), sigma=u[ok], maxfev=int(1e5))

    prediction = f.fn(pmt.R)
    pull       = (prediction - pmt.Sum) / (prediction * 1e-7)**0.5

    parametrizations[pmt_no] = f
    profiles        [pmt_no] = profile(x, y, u)
    pulls           [pmt_no] = pull

In [ ]:
plt.figure(figsize=(36, 24))

for pmt_no, pmt in df_cathode.groupby("SensorID"):
    (x, y, u), f = profiles[pmt_no], parametrizations[pmt_no]

    plt.subplot(4, 3, pmt_no + 1)
    plt.hist2d(pmt.R, pmt.Sum, (r_binning, p_binning), cmin = 1e-8)
    plt.errorbar(x, y, u, np.diff(r_binning) / 2, "--k")
    plt.plot    (x, f.fn(x), "-r")
    plt.xlabel("r (mm)")
    plt.ylabel("Detection probability")
    plt.title (f"PMT {pmt_no}")
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(36, 24))
pull_binning = np.linspace(-8, 8, 100)

for pmt_no, pmt in df_cathode.groupby("SensorID"):
    pull = pulls[pmt_no]
    y, x = np.histogram(pull, pull_binning)
    x    = x[:-1] + np.diff(x) / 2
    f    = fitf.fit(fitf.gauss, x, y, (1e4, 0, 1))

    plt.subplot(4, 3, pmt_no + 1)
    plt.hist   (pull, pull_binning, histtype="step", color="k", lw=3)
    plt.plot   (x, f.fn(x), "r-", lw=3)
    plt.text   (0.7, 0.7, f"µ = {f.values[1]:.3f}\nσ = {f.values[2]:.3f}", transform=plt.gca().transAxes, fontsize=25)
    plt.xlabel (r"$\frac{N_{p} - N_{t}}{\sqrt{N_{p}}}$")
    plt.ylabel ("Number of occurrences")
    plt.title  (f"PMT {pmt_no}")
plt.tight_layout()

# SiPM parametrization

In [ ]:
dr_binning = np.linspace(0, np.max(df_anode.dR) * 1.001, 201)
p1_binning = np.linspace(0, np.max(df_anode.P1) * 1.001, 201)

In [ ]:
parametrizations = {}
profiles         = {}
pulls            = {}
for bin_no in range(2):
    data    = df_anode[f"P{bin_no}"]
    x, y, u = fitf.profileX(df_anode.dR, data, dr_binning.size - 1, (dr_binning[0], dr_binning[-1]))
    ok      = (u > 0) & ~np.isnan(u) & np.isfinite(u)
    x, y, u = x[ok], y[ok], u[ok]
    f       = fitf.fit(fitf.polynom, x, y, (-1, +1, -1, +1, -1, +1, data.max()), sigma=u, maxfev=int(1e5))

    prediction = f.fn(data)
    pull       = (prediction - data) / (prediction * 1e-7)**0.5

    parametrizations[bin_no] = f
    profiles        [bin_no] = profile(x, y, u)
    pulls           [bin_no] = pull

In [ ]:
plt.figure(figsize=(24, 12))

for bin_no in range(2):
    (x, y, u), f = profiles[bin_no], parametrizations[bin_no]
    data         = df_anode[f"P{bin_no}"]

    plt.subplot (2, 1, bin_no + 1)
    plt.hist2d  (df_anode.dR, data, (dr_binning, p1_binning), cmin = 1e-8)
    print(x, y, u)
    plt.errorbar(x, y, u, np.diff(dr_binning) / 2, "--k")
    plt.plot    (x, f.fn(x), "-r")
    plt.xlabel  ("r (mm)")
    plt.ylabel  ("Detection probability")
    plt.title   (f"Bin {pmt_no}")
plt.tight_layout()

In [ ]:
plt.scatter(df_anode.dR, df_anode.P1)
plt.ylim(0, 1e-5)

In [ ]:
df_anode.max()

In [ ]:
for ID, df in df_anode.groupby("SensorID"):
    if ID != 12000: continue
    plt.hist2d(df.eX, df.eY, weights=df.P1, cmap="jet")
    plt.colorbar()

In [ ]:
df = df_anode[df_anode.SensorID == 12007]
plt.hist2d(df.eX, df.eY, (np.linspace(-200, 200, 401), np.linspace(-200, 200, 401)), weights=df.P1, cmap="jet")
plt.colorbar()

In [ ]:
df = df_anode[df_anode.SensorID == 14055]
plt.hist2d(df.eX, df.eY, (np.linspace(-200, 200, 401), np.linspace(-200, 200, 401)), weights=df.P1, cmap="jet")
plt.colorbar()

In [ ]:
df1 = df[:10]

In [ ]:
df1["o"] = df1.SensorID.map(sipm.X)

In [ ]:
df1

In [ ]:
sipm = pd.DataFrame.from_csv(filename_sipm_map, sep=" ", index_col=0)
sipm.X *= -1

with open(filename_sipm_map, "w") as file:
    file.write("SensorID Dice X Y\n")
    for row in sipm.

In [38]:
with open(filename_sipm_map, "w") as file:
    file.write("SensorID Dice X Y\n")
    for ID, (x, y) in SiPM_map.items():
        dice = ID // 1000
        file.write(f"{ID} {dice} {-x} {y}\n")

In [36]:
exec(open("/Users/Gonzalo/github/NEXTdata/NEWtables/NEW_maps.py").read())